In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.sparse import csr_matrix

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EmotionGCN/final_fer2013.csv')
df.head()

,emotion,pixels,Usage
0,1,89 55 24 40 43 48 53 55 59 41 33 31 22 32 42 4...,Training
1,1,204 195 181 131 50 50 57 56 66 98 138 161 173 ...,Training
2,1,18 25 49 75 89 97 100 100 101 103 105 107 107 ...,Training
3,1,49 53 44 48 45 41 28 28 36 38 39 64 78 86 74 6...,Training
4,1,0 0 0 0 0 0 0 0 0 2 0 52 190 198 213 216 220 2...,Training


In [ ]:
emotion_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
Labels = [1,3,4,6]
df = df[df.emotion.isin(Labels)]

In [ ]:
df['emotion'].value_counts()

3    7599
6    5440
4    4521
1     409
Name: emotion, dtype: int64

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)
img_array.shape

(17969, 48, 48, 1)

In [ ]:
LE = LabelEncoder()
img_labels = LE.fit_transform(df['emotion'])
img_labels = to_categorical(img_labels)
img_labels.shape

(17969, 4)

In [ ]:
LE_label_mapping = dict(zip(LE.classes_, LE.transform(LE.classes_)))
LE_label_mapping

{1: 0, 3: 1, 4: 2, 6: 3}

In [ ]:
img_labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest,y_train, y_test = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.15, random_state=42)

In [ ]:
Xtrain.shape, Xtest.shape,y_train.shape, y_test.shape

((15273, 48, 48, 1), (2696, 48, 48, 1), (15273, 4), (2696, 4))

In [ ]:
Xt, Xval, yt, yval = train_test_split(Xtrain, y_train, shuffle=True, test_size=0.15, random_state=42)

In [ ]:
Xt.shape, Xval.shape, yt.shape

((12982, 48, 48, 1), (2291, 48, 48, 1), (12982, 4))

In [ ]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.2 MB/s eta 0:00:00


In [ ]:
import cv2
import dlib
import networkx as nx
from scipy.sparse import csr_matrix

In [ ]:
from keras.models import load_model
from tensorflow.keras.models import Model
cnn_model = load_model('/content/drive/MyDrive/EmotionGCN/emote_model_newarch.h5')

In [ ]:
model_cnn = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('maxpool2d_3').output)

In [ ]:
from google.colab.patches import cv2_imshow
from scipy.spatial import cKDTree
from sklearn.metrics.pairwise import pairwise_distances
from mtcnn import MTCNN
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

In [ ]:
predictor = dlib.shape_predictor('/content/drive/MyDrive/EmotionGCN/shape_predictor_68_face_landmarks (1).dat')
landmark_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 30, 31, 33, 35,
                    36, 38, 39, 41, 42, 44, 45, 47, 48, 50, 52, 54, 55, 56, 58, 59, 61, 63, 66]
placeholder_value = 1e-5
placeholder_feature = np.full((1, 6, 6, 512), placeholder_value, dtype=np.float32)

def extract_features(img_array , predictor, labels, landmark_indices, batch_size):
    num_batches = (len(img_array) + batch_size-1) // batch_size
    print("Number of Batches: ", num_batches)
    detector = MTCNN()

    for i in range(377, 378):
        print(i)
        batch_adj = []
        batch_feat = []
        batch_labels = labels[i*batch_size:(i+1)*batch_size]

        for img in img_array[i*batch_size:(i+1)*batch_size]:
          gray = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
          gray = np.array(gray)
          gray = cv2.convertScaleAbs(gray)
          rects = detector.detect_faces(gray)

          if len(rects) == 0:
            print("No faces were detected in the image.")
            continue

          print("Rectangles: ", rects)

          landmarks = []
          for rect in rects:
            x,y,w,h = rect['box']
            dlib_rect = dlib.rectangle(x, y, x+w, y+h)
            full_landmarks = predictor(gray, dlib_rect)
            subset_landmarks = []

            for j in landmark_indices:
              if j < full_landmarks.num_parts:
                x = full_landmarks.part(j).x
                y = full_landmarks.part(j).y
                subset_landmarks.append(dlib.point(x, y))
            landmarks.append(dlib.full_object_detection(dlib_rect, dlib.points(subset_landmarks)))


          features = []
          for landmark in landmarks:
            for k in range(51):
              x, y = landmark.part(k).x, landmark.part(k).y
              patch_size = 12
              x_start = max(0, x - patch_size // 2)
              x_end = min(gray.shape[1], x + patch_size // 2)
              y_start = max(0, y - patch_size // 2)
              y_end = min(gray.shape[0], y + patch_size // 2)

              patch = gray[y_start:y_end, x_start:x_end]

              if patch is not None and patch.size != 0:
                #cv2.rectangle(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
                patch = cv2.resize(patch, (48, 48))
                patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
                patch = np.expand_dims(patch, axis=-1)
                patch = np.expand_dims(patch, axis=0)
                feature = model_cnn.predict(patch, verbose=0)
                features.append(feature)
              else:
                print(k)
                print("Skipped")
                features.append(placeholder_feature)

              #cv2_imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            break

          node_embeddings = features[0]
          for l in range(1, len(features)):
            node_embeddings = np.concatenate((node_embeddings, features[l]), axis=0)
          batch_feat.append(node_embeddings)

          num_nodes, height, width, channels = node_embeddings.shape
          flat_node_embeddings = node_embeddings.reshape(num_nodes, height * width * channels)
          print(flat_node_embeddings.shape)

          total_elements = flat_node_embeddings.size  # Total number of elements in the array
          zero_count = total_elements - np.count_nonzero(flat_node_embeddings)
          print("Number of zero values in feat:", zero_count)

          distances = pairwise_distances(flat_node_embeddings, metric='euclidean')
          mean_distance = np.mean(distances)
          max_dist = mean_distance*1.1

          kdtree = cKDTree(flat_node_embeddings)

          graph = nx.Graph()

          for m in range(len(node_embeddings)):
            graph.add_node(m, feature=node_embeddings[m])

          for a in range(len(node_embeddings)):
            indices = kdtree.query_ball_point(flat_node_embeddings[a].reshape(1, -1), r=max_dist, p=2)[0]
            for b in indices:
              if a != b and b < len(features) and not graph.has_edge(a, b) and distances[a, b] <= max_dist:
                graph.add_edge(a, b, weight=distances[a, b])

          A = nx.to_numpy_array(graph)
          A = A + np.eye(A.shape[0])
          A[A == 0] = 1e-10
          D = np.sum(A, axis=1)
          D_sqrt_inv = np.power(D, -0.5)
          D_sqrt_inv[np.isinf(D_sqrt_inv)] = 0.
          D_sqrt_inv = np.diag(D_sqrt_inv)
          A_norm = D_sqrt_inv @ A @ D_sqrt_inv
          print(A_norm.shape)
          zero_count = np.count_nonzero(A_norm)
          print("Number of zero values in A_norm:", A_norm.size - zero_count)
          batch_adj.append(A_norm)
        batch_adj = np.stack(batch_adj, axis=0)
        batch_feat = np.stack(batch_feat, axis=0)
        print("Adjacency Batch Shape is: ", batch_adj.shape)
        print("Feature Batch Shape is: " , batch_feat.shape)
        print("Batch Labels shape is: ", batch_labels.shape)
        np.savez_compressed(f'/content/drive/MyDrive/EmotionGCN/Batches/train/batch_data_{i}.npz', batch_adj=batch_adj, batch_feat=batch_feat, batch_labels=batch_labels)
        del batch_feat, batch_adj, batch_labels, node_embeddings, features, landmarks, A, graph, flat_node_embeddings, distances, kdtree, subset_landmarks, rects, gray, D,D_sqrt_inv, indices, patch, full_landmarks

In [ ]:
extract_features(Xt , predictor, yt, landmark_indices, 32)

Number of Batches:  406
377
1/1 [==============================] - 0s 236ms/step
Rectangles:  [{'box': [0, 0, 47, 55], 'confidence': 0.9607134461402893, 'keypoints': {'left_eye': (9, 11), 'right_eye': (32, 8), 'nose': (21, 19), 'mouth_left': (15, 37), 'mouth_right': (33, 36)}}]


(51, 18432)
Number of zero values in feat: 0
(51, 51)
Number of zero values in A_norm: 0
1/1 [==============================] - 0s 184ms/step
Rectangles:  [{'box': [3, 0, 41, 48], 'confidence': 0.8937321901321411, 'keypoints': {'left_eye': (16, 18), 'right_eye': (34, 18), 'nose': (26, 30), 'mouth_left': (17, 35), 'mouth_right': (33, 36)}}]
(51, 18432)
Number of zero values in feat: 0
(51, 51)
Number of zero values in A_norm: 0
1/1 [==============================] - 0s 17ms/step
Rectangles:  [{'box': [4, 0, 44, 50], 'confidence': 0.9965645670890808, 'keypoints': {'left_eye': (14, 19), 'right_eye': (34, 14), 'nose': (24, 30), 'mouth_left': (19, 39), 'mouth_right': (38, 34)}}]
(51, 18432)
Number of zero values in feat: 0
(51, 51)
Number of zero values in A_norm: 0
1/1 [==============================] - 0s 18ms/step
Rectangles:  [{'box': [3, 0, 43, 48], 'confidence': 0.9335772395133972, 'keypoints': {'left_eye': (12, 17), 'right_eye': (32, 16), 'nose': (21, 29), 'mouth_left': (14, 36), 'mo

## Model Architecture

In [ ]:
!pip install spektral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import spektral
from spektral.layers import GCNConv, GlobalMaxPool
from spektral.utils import sp_matrix_to_sp_tensor
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense, Lambda
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model
from keras.regularizers import l2

In [ ]:
X_in = Input(shape = (51, 18432))
A_in = Input(shape = (51, 51))
l2_reg = 5e-4

GCN1 = spektral.layers.GCNConv(channels = 512, activation='elu', kernel_initializer='he_normal', name='GCN1')([X_in,A_in])
B1 = BatchNormalization()(GCN1)
GCN2 = spektral.layers.GCNConv(channels = 256, activation='elu', kernel_initializer='he_normal', name='GCN2')([B1, A_in])
B2 = BatchNormalization()(GCN2)
D1 = Dropout(0.4)(B2)
GCN3 = spektral.layers.GCNConv(channels = 256, activation='elu', kernel_initializer='he_normal', name='GCN3')([D1,A_in])
B3 = BatchNormalization()(GCN3)
GCN4 = spektral.layers.GCNConv(channels = 128 ,activation='elu', kernel_initializer='he_normal', name='GCN4')([B3, A_in])
B4 = BatchNormalization()(GCN4)
D2 = Dropout(0.4)(B4)
GCN5 = spektral.layers.GCNConv(channels = 128,activation='elu', kernel_initializer='he_normal', name='GCN5')([D2, A_in])
B5 = BatchNormalization()(GCN5)
GCN6 = spektral.layers.GCNConv(channels = 64 ,activation='elu', kernel_initializer='he_normal',name='GCN6')([B5, A_in])
B6 = BatchNormalization()(GCN6)
MP1 = spektral.layers.GlobalMaxPool()(B6)
D3 = Dropout(0.4)(MP1)
flatten = Flatten()(D3)
DS1 = Dense(64 ,activation='elu',kernel_initializer='he_normal')(flatten)
B7 = BatchNormalization()(DS1)
DS2 = Dense(32 ,activation='elu',kernel_initializer='he_normal')(B7)
B8 = BatchNormalization()(DS2)
D4 = Dropout(0.4)(B8)
output = Dense(4, activation='softmax')(D4)

model = Model(inputs=[X_in, A_in], outputs=output)

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate = 0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 51, 18432)]          0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 51, 51)]             0         []                            
                                                                                                  
 GCN1 (GCNConv)              (None, 51, 512)              9437696   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 51, 512)              2048      ['GCN1[0][0]']            

In [ ]:
import random
def train_gen():
  num_batches = 406
  print("Number of Batches: ", num_batches)

  batch_indices = list(range(num_batches))

  while True:
    random.shuffle(batch_indices)

    for k in batch_indices:
      filename = f'/content/drive/MyDrive/EmotionGCN/Batches/train/batch_data_{k}.npz'
      data = np.load(filename)
      batch_adj = data['batch_adj']
      batch_feat = data['batch_feat']
      batch_feat = batch_feat.reshape(batch_feat.shape[0], batch_feat.shape[1], -1)
      batch_labels = data['batch_labels']

      yield (batch_feat, batch_adj), batch_labels
      del batch_feat, batch_adj, batch_labels, data

In [ ]:
def val_gen():
  num_batches = 72
  print("Number of Batches: ", num_batches)

  batch_indices = list(range(num_batches))

  while True:
    random.shuffle(batch_indices)

    for k in batch_indices:
      data = np.load(f'/content/drive/MyDrive/EmotionGCN/Batches/val/batch_data_{k}.npz')
      batch_adj = data['batch_adj']
      batch_feat = data['batch_feat']
      batch_feat = batch_feat.reshape(batch_feat.shape[0], batch_feat.shape[1], -1)
      batch_labels = data['batch_labels']

      yield (batch_feat, batch_adj), batch_labels
      del batch_feat, batch_adj, batch_labels, data

## Data Augmentation

In [ ]:
train_datagenerator = ImageDataGenerator(rotation_range=15,width_shift_range=0.15,height_shift_range=0.15,
    shear_range=0.15,zoom_range=0.15,horizontal_flip=True)

## Model Fitting

In [ ]:
model_path = '/content/drive/MyDrive/EmotionGCN/model_05_1.04_0.53.h5'

In [ ]:
tf.keras.utils.get_custom_objects()['GCNConv'] = GCNConv
tf.keras.utils.get_custom_objects()['GlobalMaxPool'] = GlobalMaxPool
model = tf.keras.models.load_model(model_path)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=10,verbose=1,
    restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=7,min_lr=1e-7,verbose=1)
filepath = "/content/drive/MyDrive/EmotionGCN/model_{epoch:02d}_{val_loss:.2f}_{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
callbacks = [early_stopping,lr_scheduler, checkpoint]

In [ ]:
import numpy as np

In [ ]:
batch_size = 32
epochs = 75
train_generator = train_gen()
val_generator = val_gen()
history = model.fit(train_generator, batch_size = batch_size, epochs = epochs, steps_per_epoch = 406, validation_data = val_generator, validation_steps= 72,callbacks=callbacks)

Number of Batches:  406
Epoch 1/75
406/406 [==============================] - ETA: 0s - loss: 1.0395 - accuracy: 0.5297Number of Batches:  72

Epoch 1: val_loss improved from inf to 1.32682, saving model to /content/drive/MyDrive/EmotionGCN/model_01_1.33_0.46.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


406/406 [==============================] - 823s 2s/step - loss: 1.0395 - accuracy: 0.5297 - val_loss: 1.3268 - val_accuracy: 0.4561 - lr: 0.0010
Epoch 2/75
406/406 [==============================] - ETA: 0s - loss: 1.0184 - accuracy: 0.5419
Epoch 2: val_loss did not improve from 1.32682
406/406 [==============================] - 742s 2s/step - loss: 1.0184 - accuracy: 0.5419 - val_loss: 1.3291 - val_accuracy: 0.3645 - lr: 0.0010
Epoch 3/75
406/406 [==============================] - ETA: 0s - loss: 0.9904 - accuracy: 0.5659
Epoch 3: val_loss improved from 1.32682 to 1.21846, saving model to /content/drive/MyDrive/EmotionGCN/model_03_1.22_0.50.h5
406/406 [==============================] - 750s 2s/step - loss: 0.9904 - accuracy: 0.5659 - val_loss: 1.2185 - val_accuracy: 0.4987 - lr: 0.0010
Epoch 4/75
406/406 [==============================] - ETA: 0s - loss: 0.9734 - accuracy: 0.5772
Epoch 4: val_loss did not improve from 1.21846
406/406 [==============================] - 729s 2s/step - l